In [ ]:
import kwant
import functools as ft
import matplotlib.pyplot as plt
import numpy as np
import adaptive
adaptive.notebook_extension()

import scipy.constants
import cmath

import os, sys
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))
import sns_system
import supercurrent_matsubara
import supercurrent

import distributed_sns as ds
import holoviews as hv

In [ ]:
syst_pars = dict(
    L_m = 500,
    L_x = 4800,
    L_sc_up = 200,
    L_sc_down = 200,
    z_x = 2400,
    z_y = 400,
    a   = 10,
    parallel_curve = False,
    sawtooth=True,
    transverse_soi = True,
    mu_from_bottom_of_spin_orbit_bands = True,
    k_x_in_sc = True,
    wraparound = False,
    current = False,
    ns_junction=False)

In [ ]:
%%time
syst, site_colors, hopping = sns_system.make_system(**syst_pars)

In [ ]:
coloring =  {'middle_interior' : 'grey',
             'middle_barrier' : 'black',
             'bottom_superconductor' : 'gold',
             'top_superconductor' : 'gold',
             'top_cut' : 'red',
             'bottom_cut' : 'blue'}

for k,v in site_colors.items():
    if site_colors.get(k) is not None:
        site_colors[k] = coloring[v]

In [ ]:
kwant.plot(syst,site_color=sns_system.site_color_function(site_colors, syst), dpi=400);

In [ ]:
constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

params_raw= dict(g_factor_middle = 26,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 10,
                 alpha_middle = 20,
                 alpha_left = 0,
                 alpha_right = 0,
                 Delta_left = 2,
                 Delta_right = 2,
                 B = .3,
                 phase = np.pi,
                 T = 0.0,
                 V = 0)

params = dict(**constants,
              **params_raw)

In [ ]:
keys_with_bounds = {'B' : [0, 1],
                    'phase' : [0, 2*np.pi]}
metric_params = {'lowest_two_energies' : {}}

In [ ]:
agg_SS = ds.AggregatesSimulationSet(keys_with_bounds, syst_pars, params, metric_params,'lowest_two_energies')

In [ ]:
z_ys = [100, 200, 300, 400, 500, 600]
def _params_z_y(syst_pars, params, z_y):
    for key in ['Delta_left', 'Delta_right']:
        if key in params:
            params[key] = z_y
        if key in syst_pars:
            syst_pars[key] = z_y
    return z_y

params_z_y = [ft.partial(_params_z_y,
                           z_y=_z_y
                           ) for _z_y in z_ys]
agg_SS.add_dimension('z_y', params_z_y)

In [ ]:
agg_SS.make_balancing_learner(300)

In [ ]:
bl = agg_SS.get_balancing_learner()

In [ ]:
runner = adaptive.Runner(bl, ntasks=4, goal = lambda l: l.loss()<0.5)
runner.live_info()

In [ ]:
hv.notebook_extension('matplotlib')

In [ ]:
%%opts Image [colorbar=True aspect=1] (cmap='Viridis') {+framewise}
kdims, pdict = agg_SS.get_plot_dict(200, False, False)
hv.HoloMap(pdict, kdims=kdims)

In [ ]:
hv.renderer('matplotlib').save(res, 'test', fmt='gif')

In [ ]:
_ = bl.learners[0]

In [ ]:
[(l.npoints, l.loss()) for l in bl.learners]





In [ ]:
_ = hv.renderer('matplotlib')

In [ ]:
_.save()